# Ship360 Chat API - Fundamentals

This notebook introduces the core concepts and usage patterns of the Ship360 Chat API. You'll learn how to initialize the API client, make basic queries, and understand the structure of responses.

## Overview

The Ship360 Chat API provides a conversational interface for shipping-related queries. It allows users to:

1. Track shipments
2. Get shipping rates
3. Find delivery options
4. Troubleshoot shipping issues
5. Get support for shipping-related questions

This API leverages Azure OpenAI and Semantic Kernel to provide intelligent responses to natural language queries.

## Required Dependencies

Before we begin, let's ensure we have all the necessary libraries installed.

In [ ]:
# Install required packages
# Uncomment and run if needed
# !pip install requests
# !pip install python-dotenv
# !pip install pydantic

## Importing Libraries

First, let's import the necessary libraries for making API requests and handling responses.

In [ ]:
import os
import json
import requests
from dotenv import load_dotenv
from pprint import pprint

# Load environment variables
load_dotenv()

# Define the base URL for the API
BASE_URL = "http://localhost:8000"

## Environment Setup

The API requires certain environment variables to be set. We'll create a `.env` file in the `docs/examples/notebooks/` directory with the following variables. This keeps our example notebooks isolated from the main application.

In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Set path to the .env file in the notebooks directory
notebooks_dir = Path(os.path.abspath('')).parent.parent
env_path = notebooks_dir / '.env'

# Load environment variables from the notebooks/.env file
load_dotenv(dotenv_path=env_path)

# Set up environment variables
# You can also create a .env file with the following variables
# AZURE_OPENAI_API_KEY=your-api-key
# AZURE_OPENAI_ENDPOINT=your-endpoint

# Check if environment variables are set
def check_env_vars():
    required_vars = ["AZURE_OPENAI_API_KEY", "AZURE_OPENAI_ENDPOINT"]
    missing_vars = [var for var in required_vars if not os.environ.get(var)]
    
    if missing_vars:
        print(f"Warning: Missing environment variables: {', '.join(missing_vars)}")
        print(f"Please create a .env file at {env_path} with these variables.")
        print("Some examples in this notebook may not work without these variables.")
    else:
        print("All required environment variables are set!")

# Uncomment to check environment variables
# check_env_vars()

## API Endpoints

The Ship360 Chat API provides several endpoints for different functionalities. Here are the main endpoints we'll be working with:

1. `/api/v1/chat` - For sending chat messages and getting responses
2. `/api/v1/orders` - For querying order information

Let's explore these endpoints one by one.

## Making Basic API Calls

Let's start by creating a simple function to send requests to the Ship360 Chat API.

In [ ]:
def send_chat_message(message, endpoint="/api/v1/chat"):
    """Send a chat message to the API and return the response."""
    url = f"{BASE_URL}{endpoint}"
    headers = {"Content-Type": "application/json"}
    payload = {"message": message}
    
    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()  # Raise an exception for 4XX/5XX responses
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error making request: {e}")
        return None

## Example: Sending a Simple Query

Let's try sending a simple query to the API. For this example, we'll ask about tracking a shipment.

In [ ]:
# Example query
response = send_chat_message("Where is my shipment with tracking number 1Z999AA10123456784?")

# Pretty print the response
if response:
    print("API Response:")
    pprint(response)
else:
    print("Failed to get a response from the API. Make sure the API server is running.")

## Understanding API Responses

The API responses are in JSON format and typically include:

1. `response` - The main text response to the user's query
2. `intent` - The detected intent of the user's query
3. `details` - Additional structured data related to the query (if available)

Let's create a function to parse and display these components in a more readable format.

In [ ]:
def parse_api_response(response):
    """Parse and display components of the API response."""
    if not response:
        print("No response to parse.")
        return
    
    print("\n==== Chat Response ====")
    print(response.get("response", "No text response available."))
    
    print("\n==== Detected Intent ====")
    print(response.get("intent", "No intent detected."))
    
    print("\n==== Additional Details ====")
    details = response.get("details", {})
    if details:
        pprint(details)
    else:
        print("No additional details available.")
        
# Parse the previous response
if 'response' in locals() and response:
    parse_api_response(response)

## Error Handling

When working with APIs, it's important to handle errors gracefully. Let's enhance our API call function to include better error handling.

In [ ]:
def send_chat_message_with_error_handling(message, endpoint="/api/v1/chat", timeout=10):
    """Send a chat message with improved error handling."""
    url = f"{BASE_URL}{endpoint}"
    headers = {"Content-Type": "application/json"}
    payload = {"message": message}
    
    try:
        response = requests.post(url, headers=headers, json=payload, timeout=timeout)
        
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: HTTP {response.status_code}")
            print(f"Error details: {response.text}")
            return None
            
    except requests.exceptions.ConnectionError:
        print("Connection error. Make sure the API server is running.")
        return None
    except requests.exceptions.Timeout:
        print(f"Request timed out after {timeout} seconds.")
        return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

## Advanced Example: Querying Order Information

Let's try a more advanced example: querying information about a specific order.

In [ ]:
# Example order query
order_query = send_chat_message_with_error_handling("What's the status of my order #12345?")

# Parse and display the response
if order_query:
    parse_api_response(order_query)

## Handling Multiple Intents

Users might ask complex questions that involve multiple intents. Let's see how the API handles this.

In [ ]:
# Example multi-intent query
multi_intent_query = send_chat_message_with_error_handling(
    "I need to know the shipping rate for a 5lb package to California, and also where my order #54321 is."
)

# Parse and display the response
if multi_intent_query:
    parse_api_response(multi_intent_query)

## Shipping Rate Calculations

One common use case is calculating shipping rates. Let's explore how to use the API for this.

In [ ]:
# Example shipping rate query
shipping_rate_query = send_chat_message_with_error_handling(
    "How much would it cost to ship a 3lb package from Seattle to Miami?"
)

# Parse and display the response
if shipping_rate_query:
    parse_api_response(shipping_rate_query)

## Conclusion

In this notebook, we've explored the basics of working with the Ship360 Chat API:

1. Setting up the environment
2. Making basic API calls
3. Understanding API responses
4. Handling errors
5. Working with specific use cases like order tracking and shipping rate calculations

This foundation will help you build more complex applications that leverage the Ship360 Chat API for shipping-related functionality.

## Next Steps

- Explore more advanced features like multi-turn conversations
- Implement authentication for production use
- Create custom UIs that integrate with the API
- Combine multiple API calls to create comprehensive shipping solutions